# day 14

https://adventofcode.com/14/day/14

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day14.txt')

LOGGER = logging.getLogger('day14')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#...."""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

In [ ]:
import numpy as np


def parse_data(d: str) -> np.ndarray:
    return np.array([list(row.strip())
                     for row in d.strip().split('\n')])

parse_data(test_data)

#### function def

In [ ]:
def find_cube_rock_north(i: int, j: int, a: np.ndarray) -> int:
    try:
        return (a[:i + 1, j] == '#').nonzero()[0].max()
    except ValueError:
        return -1

a = parse_data(test_data)
for i in range(7):
    assert find_cube_rock_north(i=i, j=0, a=a) == -1, str(i)
assert find_cube_rock_north(i=8, j=0, a=a) == 8
assert find_cube_rock_north(i=9, j=0, a=a) == 9

In [ ]:
def count_round_rocks_above(i: int, j: int, i_cube: int, a: np.ndarray) -> int:
    return (a[max(i_cube, 0): i, j] == 'O').sum()

a = parse_data(test_data)
assert count_round_rocks_above(i=0, j=0, i_cube=0, a=a) == 0
assert count_round_rocks_above(i=1, j=0, i_cube=0, a=a) == 1
assert count_round_rocks_above(i=2, j=0, i_cube=0, a=a) == 2
assert count_round_rocks_above(i=3, j=0, i_cube=0, a=a) == 2
assert count_round_rocks_above(i=4, j=0, i_cube=0, a=a) == 3
assert count_round_rocks_above(i=5, j=0, i_cube=0, a=a) == 3
assert count_round_rocks_above(i=6, j=0, i_cube=0, a=a) == 4

assert count_round_rocks_above(i=9, j=0, i_cube=5, a=a) == 1

In [ ]:
def calculate_load(i: int, j: int, a: np.ndarray) -> int:
    c = a[i, j]
    if c in '.#':
        return 0

    i_cube = find_cube_rock_north(i=i, j=j, a=a)
    n_round = count_round_rocks_above(i=i, j=j, i_cube=i_cube, a=a)
    row = i_cube + 1 + n_round
    return a.shape[1] - row

assert calculate_load(i=0, j=0, a=a) == 10
assert calculate_load(i=1, j=0, a=a) == 9
assert calculate_load(i=3, j=0, a=a) == 8
assert calculate_load(i=6, j=2, a=a) == 4
assert calculate_load(i=9, j=2, a=a) == 3

assert calculate_load(i=2, j=0, a=a) == 0
assert calculate_load(i=8, j=0, a=a) == 0

In [ ]:
def q_1(data):
    a = parse_data(d=data)
    return sum(calculate_load(i=i, j=j, a=a) for i in range(a.shape[0]) for j in range(a.shape[1]))

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 136
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
parse_data(test_data)

In [ ]:
from functools import cache

sort_order = {'O': 0, '.': 1, '#': 2}

@cache
def sort_vec(v_str: str, reverse: bool = False) -> np.ndarray:
    pieces = v_str.split('#')
    sorted_pieces = [''.join(sorted(substr, key=lambda x: (-1 if reverse else 1) * sort_order[x]))
                     for substr in pieces]
    return np.array(list('#'.join(sorted_pieces)))

assert (sort_vec('OO.O.O..##') == (np.array(list('OOOO....##')))).all()
assert (sort_vec('...OO....O') == (np.array(list('OOO.......')))).all()
assert (sort_vec('#.#..O#.##') == (np.array(list('#.#O..#.##')))).all()

assert (sort_vec('OO.O.O..##', reverse=True) == (np.array(list('....OOOO##')))).all()
assert (sort_vec('...OO....O', reverse=True) == (np.array(list('.......OOO')))).all()
assert (sort_vec('#.#..O#.##', reverse=True) == (np.array(list('#.#..O#.##')))).all()

In [ ]:
from enum import Enum


class TiltDir(Enum):
    N = 'N'
    E = 'E'
    S = 'S'
    W = 'W'


class Dish:
    def __init__(self, a: np.ndarray):
        self.a = a

    def as_str(self) -> str:
        return '\n'.join(''.join(char for char in row) for row in self.a)

    def tilt(self, tilt_dir: TiltDir) -> None:
        if tilt_dir in [TiltDir.N, TiltDir.S]:
            # iterate through columns
            for j in range(self.a.shape[1]):
                v_str = ''.join(self.a[:, j])
                self.a[:, j] = sort_vec(v_str=v_str, reverse=tilt_dir is TiltDir.S)
        elif tilt_dir in [TiltDir.E, TiltDir.W]:
            # iterate through rows
            for i in range(self.a.shape[0]):
                v_str = ''.join(self.a[i, :])
                self.a[i, :] = sort_vec(v_str=v_str, reverse=tilt_dir is TiltDir.E)

    def find_loop(self):
        self.loop_dict = {}
        i = 0
        have_found_loop = False
        while not have_found_loop:
            for tilt_dir, next_tilt_dir in [(TiltDir.N, TiltDir.W),
                                            (TiltDir.W, TiltDir.S),
                                            (TiltDir.S, TiltDir.E),
                                            (TiltDir.E, TiltDir.N)]:
                k = self.as_str(), tilt_dir
                self.tilt(tilt_dir=tilt_dir)
                next_k = self.as_str(), next_tilt_dir
                v_update = {'step': i, 'next_k': next_k}
                try:
                    self.loop_dict[k].append(v_update)
                    have_found_loop = True
                    break
                except KeyError:
                    self.loop_dict[k] = [v_update]
                i += 1

    @property
    def cycle_key(self) -> tuple[str, TiltDir]:
        try:
            return [k for (k, v) in self.loop_dict.items() if len(v) > 1][0]
        except AttributeError:
            raise AttributeError("have you run `.find_loop` yet?")

    @property
    def loop_steps(self) -> list[int]:
        k = self.cycle_key
        return list(sorted(_['step'] for _ in self.loop_dict[k]))

In [ ]:
a = parse_data(test_data)
d = Dish(a=a)

assert d.as_str() == test_data

d.tilt(tilt_dir=TiltDir.N)
assert d.as_str() == """OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#...."""

d.tilt(tilt_dir=TiltDir.W)
assert d.as_str() == """OOOO.#O...
OO..#....#
OOO..##O..
O..#OO....
........#.
..#....#.#
O....#OO..
O.........
#....###..
#....#...."""

d.tilt(tilt_dir=TiltDir.S)
assert d.as_str() == """.....#....
....#.O..#
O..O.##...
O.O#......
O.O....O#.
O.#..O.#.#
O....#....
OO....OO..
#O...###..
#O..O#...."""

d.tilt(tilt_dir=TiltDir.E)
assert d.as_str() == """.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#...."""

In [ ]:
a = parse_data(test_data)
d = Dish(a=a)
d.find_loop()

In [ ]:
d.loop_steps

In [ ]:
a = parse_data(test_data)
d = Dish(a=a)

d.tilt(tilt_dir=TiltDir.N)
d.tilt(tilt_dir=TiltDir.W)
d.tilt(tilt_dir=TiltDir.S)
d.tilt(tilt_dir=TiltDir.E)
assert d.as_str(), """.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#...."""

d.tilt(tilt_dir=TiltDir.N)
d.tilt(tilt_dir=TiltDir.W)
d.tilt(tilt_dir=TiltDir.S)
d.tilt(tilt_dir=TiltDir.E)
assert d.as_str(), """.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O"""

d.tilt(tilt_dir=TiltDir.N)
d.tilt(tilt_dir=TiltDir.W)
d.tilt(tilt_dir=TiltDir.S)
d.tilt(tilt_dir=TiltDir.E)
assert d.as_str(), """.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O"""

In [ ]:
def get_score(a: np.ndarray) -> int:
    n = a.shape[0]
    return ((a == 'O').sum(axis=1) * range(n, 0, -1)).sum()

a = parse_data("""OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....""")
assert get_score(a) == 136

In [ ]:
def q_2(data):
    a = parse_data(data)
    d = Dish(a=a)

    d.find_loop()
    loop_start, loop_end = d.loop_steps
    loop_len = loop_end - loop_start

    n_cycles = 1_000_000_000
    n_steps = n_cycles * 4
    point_in_loop = (n_steps - loop_start) % loop_len
    new_step = loop_start + point_in_loop

    assert new_step % 4 == 0

    for (k, v) in d.loop_dict.items():
        if v[0]['step'] == new_step:
            s = k[0]

    return get_score(parse_data(s))

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 64
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin